# NeuroCluster:
<font size= 4> A Python toolbox for nonparametric cluster-based statistical testing of neurophysiological data with respect to continuous predictors 

First Authors: Alexandra Fink-Skular & Christina Maher  \
Updated: 11/10/2024 by AFS

In [1]:
# load required libraries
import numpy as np
import pandas as pd
from glob import glob
import datetime
from IPython import display

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import NeuroCluster

In [ ]:
# If you are running NeuroCluster from a local repo (not through a virtual environment package install)
user_base_dir   = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/neuro_cluster_proj/'
# user_base_dir   = '/Path/To/NeuroCluster/NeuroCluster/'
sample_data_dir = f'{user_base_dir}NeuroCluster/data/'
results_dir     = f'{user_base_dir}NeuroCluster/results/'

# Let's store the date so we can keep track of versions
date = datetime.date.today().strftime('%m%d%Y')

In [ ]:
# load custom functions
import sys
sys.path.append(f'{user_base_dir}NeuroCluster/')
import NeuroCluster

# Neurocluster Workflow

This notebook performs a nonparametric permutation-based cluster test on continuous behavioral predictors for one example electrode. The workflow is outlined schematically below.

In [6]:
from IPython.display import Image, display
import os 

# Get the base path of the installed NeuroCluster package
package_dir = os.path.dirname(NeuroCluster.__file__)

# Construct the path to the image
image_path = os.path.join(package_dir, "workflow", "Figure1_workflow.png")

# Display the image
display(Image(filename=image_path, width=750))

FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/envs/neurocluster_env/lib/python3.10/site-packages/NeuroCluster/workflow/Figure1_workflow.png'

# Step 1: Format input data (neural and behavioral)

The sample data for this notebook includes: 
- neural data: np.array (n_channels x n_trials x n_freqs x n_times)
- behavior data: pd.DataFrame (n_trials x n_variables)

These variables are extracted from an `mne.time_frequency.EpochsTFR` which is a spatiotemporal representation of neural data that includes power across trials, frequencies, and timepoints. Note, for this notebook we provide the neural and behavioral data in `np.array` *a priori*. These data can be found in the `sample_data_dir`.

Here is some example code, loading an `EpochsTFR` following preprocessing with  [MNE-Python](https://mne.tools/stable/index.html) and creating a neural data file (i.e., `channel_13.npy`) and our behavioral regressors (i.e., `sample_behavior.csv`):

`# load EpochsTFR data`


`power_epochs = mne.time_frequency.read_tfrs(fname=f'{tfr_dir}{subj_id}_tfr.h5')[0]`



`# save data from each channel to a .npy file`


`for i in range(len(power_epochs.info['ch_names'])):`

        channel = power_epochs.info['ch_names'][i]

        data = power_epochs.data[i]

        np.save(f'{results_dir}{channel}.npy', data)



`# save metadata as a numpy array to a .csv file`


`metadata = pd.DataFrame(power_epochs.metadata)`


`metadata.to_csv(f'{results_dir}sample_behavior.csv', index=False)`



In the following cell, let's load our template neural data into a dictionary and load the DataFrame of our behavioral predictors.

In [ ]:
sample_ieeg_files = glob(f'{sample_data_dir}*.npy')
sample_ieeg_dict  = {f'{file.split("/")[-1].split(".")[0]}':np.load(file) for file in sample_ieeg_files}
ieeg_channels     = list(sample_ieeg_dict.keys())
sample_behav      = pd.read_csv(f'{sample_data_dir}sample_behavior.csv')
freqs             = np.logspace(*np.log10([2, 200]), num=30) # this information should be stored in the metadata of your neural data recording. 

Let's define the behavioral variables we plan to include as independent variables in our linear regression (`multi_reg_vars`) and the regressor of interest (`target_var`) we will permute to determine whether significant clusters encoding this behavioral variable exist in our time x frequency data. All continuous predictors should be normalized. 

In [ ]:
# set main predictor of interest for permutations *target_var must be a continuous numeric variable*
target_var = 'error'

# define subset of predictor variables from sample_behav to include in regression (should include target_var)
multi_reg_vars = ['outcome','error']

# subset input dataframe to include only multi_reg_vars
predictor_data = sample_behav.copy()[multi_reg_vars]

# let's print the first few rows of the predictor data to make sure it looks right
predictor_data.head()


# Step 2: Perform within-electrode cluster test.

First, let's create a variable called `tfr_data` which is a `np.array` (dimensions should correspond with number of trials x number of frequencies x number of timepoints)

In [ ]:
# subset demo channel data from sample_ieeg_dict and store as tfr_data variable: np.array of (num epochs x num frequencies x num times)
demo_channel = 'channel_4'
tfr_data     = sample_ieeg_dict[demo_channel]

# check tfr_data dimensions - must be num trials, num frequencies, num timepoints
tfr_data.shape 
print(f'Number of trials for {demo_channel}: {tfr_data.shape[0]}')
print(f'Number of frequencies for {demo_channel}: {tfr_data.shape[1]}')
print(f'Number of timepoints for {demo_channel}: {tfr_data.shape[2]}')

Next, let's create an instance of `TFR_Cluster_Test`. This will be used to run the cluster test. It requires the tfr_data, predictor_data, target_var, and demo_channel as inputs.

In [ ]:
help(NeuroCluster.TFR_Cluster_Test)

In [ ]:
cluster_test  = NeuroCluster.TFR_Cluster_Test(tfr_data,predictor_data,target_var,demo_channel)
cluster_test

To calculate t-critical for a two-sided hypothesis test, we compute a T-distribution with N-K-1 degrees of freedom (N=number of samples, K = number of predictors in regression model) and find the t-values where the area of the t-distribution = 0.025 and 0.975 (1-alpha/ntails,alpha=0.05,ntails=2). 

In [ ]:
NeuroCluster.plotting_utils.plot_tcritical(cluster_test)

Now, we are reading to run our linear regression based on the dependent neural variable (`tfr_data`) and independent behavioral variables (`predictor_data`, `target_var`) we passed as inputs to our `TFR_Cluster_Test` object. This will return pixel-level **β coefficients** and corresponding **t-statistics** for our TFR data in one electrode.

In [ ]:
betas, tstats = cluster_test.tfr_regression()

Let's plot the **β coefficients** to give us an idea of the neural encoding pattern for our continuous predictor of interest.

In [ ]:
NeuroCluster.plot_beta_coef(betas,cluster_test,freqs)

We can also plot the **t-statistics** that correspond with the **β coefficient** for each time-frequency point.

In [ ]:
NeuroCluster.plot_tstats(tstats,cluster_test,freqs)

Let's highlight **clusters** (defined as consecutive time x frequency points) with significant t-statistics. We can do this separately for both positive and negative clusters. 

In [ ]:
NeuroCluster.plot_clusters(tstats,cluster_test,freqs)

# Step 3: Calculate True Cluster Statistic(s) 

Now, we will identify the largest cluster (either/both positive and negative) and save the **cluster statistic** which will be our test statistic against our non-parametric null distribution. `max_tfr_cluster()` returns a dictionary containing the **cluster statistic**:`cluster_stat` and its associated **freq_idx**: `freq_idx` and **time_idx**:`time_idx`.

In [ ]:
# Step 3: Find largest cluster(s) and return the max cluster statistic(s) and cluster's  frequencies x times indices
max_cluster_data  = cluster_test.max_tfr_cluster(tstats,max_cluster_output='all')
print(f'Max positive cluster dictionary: {max_cluster_data[0]}')
print(f'Max negative cluster dictionary: {max_cluster_data[1]}')

Let's plot our largest cluster and its associated **cluster statistic**

In [ ]:
NeuroCluster.plot_max_clusters(cluster_test,tstats,freqs)
# TFR-Level Test Statistic: Largest Cluster

# Step 4: Compute cluster p-value(s) from null distribution of cluster statistics. 
To generate the null distribution, perform non-parametric cluster-based permutation testing by randomly permuting predictor of interest (target_var). 

At this point, we have computed the true cluster statistics from our neural (`tfr_data`) and behavioral data (`predictor_data`, `target_var`). Next, we will permute our input data and re-run the cluster identification procedure on each permuted dataset. This will allow us to generate a null distribution of cluster statistics, which we can use to evaluate the statistical significance of the cluster statistics observed in our true data.

`compute_null_cluster_stats()` takes `num_permutations` as an input, which specifies the desired number of permutations. The function will permute the regressor of interest according to this number. It returns a list of null cluster statistics, with the length of the list depending on the tails of the test. Here we generated 100 null cluster statistics, `num_permutations=100`, but we recommend running at least 200 permutations (500 to 1000 is best practice). 

In [ ]:
null_cluster_distribution = cluster_test.compute_null_cluster_stats(num_permutations=100)

In [ ]:
null_cluster_distribution

We will compute the ***p*-value** associated with our true cluster statistics based on the null distributions we create using `cluster_significance_test()`.

In [ ]:
cluster_pvalue = cluster_test.cluster_significance_test(max_cluster_data,null_cluster_distribution) 
print(f'Positive cluster p-value: {cluster_pvalue[0]}')
print(f'Negative cluster p-value: {cluster_pvalue[1]}')

Let's create a plot showing the **null distribution(s)** we generated, with our true cluster statistic overlaid on top.

In [ ]:
NeuroCluster.plot_null_distribution(null_cluster_distribution, max_cluster_data,cluster_pvalue,dpi=125)

In [ ]:
# Save all plots
tstat_threshold = cluster_test.threshold_tfr_tstat(tstats)
tcrit_plot,beta_plot,tstat_plot,cluster_plot,max_cluster_plot,null_distribution_plot = NeuroCluster.plot_neurocluster_results(betas,cluster_test,
                                                                                                                    max_cluster_data, null_cluster_distribution, tstats, tstat_threshold,cluster_pvalue,freqs)

# Define the directory where you want to save the plots
output_directory = f'{results_dir}/{demo_channel}_{target_var}'

# Create the directory if it doesn't exist
NeuroCluster.create_directory(output_directory)

# Save plots to the output directory
NeuroCluster.save_plot_to_pdf(tcrit_plot, output_directory, f'{cluster_test.alternative}_tcrit_plot.pdf')
NeuroCluster.save_plot_to_pdf(beta_plot, output_directory, f'{cluster_test.alternative}_beta_plot.pdf')
NeuroCluster.save_plot_to_pdf(tstat_plot, output_directory, f'{cluster_test.alternative}_tstat_plot.pdf')
NeuroCluster.save_plot_to_pdf(cluster_plot, output_directory, f'{cluster_test.alternative}_cluster_plot.pdf')
NeuroCluster.save_plot_to_pdf(max_cluster_plot, output_directory, f'{cluster_test.alternative}_max_cluster_plot.pdf')
NeuroCluster.save_plot_to_pdf(null_distribution_plot, output_directory, f'{cluster_test.alternative}_null_distribution_plot.pdf')


# One-Sided Hypothesis Test Example
Rather than testing whether a tfr cluster significantly encodes our `target_var` in general, we can evaluate the directionality of `target_var` encoding in our cluster. Specifically, we can test whether neuronal activity in a cluster increases (or decreases) with increasing (or decreasing) values of the `target_var`.

In [ ]:
cluster_test  = NeuroCluster.TFR_Cluster_Test(tfr_data,predictor_data,target_var,demo_channel,alternative='greater')
cluster_test

To calculate t-critical for a one-sided hypothesis test, we compute a T-distribution with N-K-1 degrees of freedom (N=number of samples, K = number of predictors in regression model) and find the t-value where the area of the t-distribution = 0.95 (1-alpha,alpha=0.05). 


In [ ]:
NeuroCluster.plotting_utils.plot_tcritical(cluster_test)

Now, we are reading to run our linear regression based on the dependent neural variable (`tfr_data`) and independent behavioral variables (`predictor_data`, `target_var`) we passed as inputs to our `TFR_Cluster_Test` object. This will return pixel-level **β coefficients** and corresponding **t-statistics** for our TFR data in one electrode.

In [ ]:
betas, tstats = cluster_test.tfr_regression()

Let's plot the **β coefficients** to give us an idea of the neural encoding pattern for our continuous predictor of interest.

In [ ]:
NeuroCluster.plot_beta_coef(betas,cluster_test,freqs)

We can also plot the **t-statistics** that correspond with the **β coefficient** for each time-frequency point.

In [ ]:
NeuroCluster.plot_tstats(tstats,cluster_test,freqs)

Let's highlight **clusters** (defined as consecutive time x frequency points) with significant t-statistics.

In [ ]:
NeuroCluster.plot_clusters(tstats,cluster_test,freqs,figsize=(6,5))

# Step 3: Calculate True Cluster Statistic(s) 

Now, we will identify the largest cluster (either/both positive and negative) and save the **cluster statistic** which will be our test statistic against our non-parametric null distribution. `max_tfr_cluster()` returns a dictionary containing the **cluster statistic**:`cluster_stat` and its associated **freq_idx**: `freq_idx` and **time_idx**:`time_idx`.

In [ ]:
# Step 3: Find largest cluster(s) and return the max cluster statistic(s) and cluster's  frequencies x times indices
max_cluster_data  = cluster_test.max_tfr_cluster(tstats,max_cluster_output='all')
print(f'Max positive cluster dictionary: {max_cluster_data[0]}')

Let's plot our **positive** largest cluster and its associated **cluster statistic**

In [ ]:
NeuroCluster.plot_max_clusters(cluster_test,tstats,freqs,figsize=(6,5))

# Step 4: Compute cluster p-value(s) from null distribution of cluster statistics. 
To generate the null distribution, perform non-parametric cluster-based permutation testing by randomly permuting predictor of interest (target_var). 

At this point, we have computed the true cluster statistics from our neural (`tfr_data`) and behavioral data (`predictor_data`, `target_var`). Next, we will permute our input data and re-run the cluster identification procedure on each permuted dataset. This will allow us to generate a null distribution of cluster statistics, which we can use to evaluate the statistical significance of the cluster statistics observed in our true data.

`compute_null_cluster_stats()` takes `num_permutations` as an input, which specifies the desired number of permutations. The function will permute the regressor of interest according to this number. It returns a list of null cluster statistics, with the length of the list depending on the tails of the test. Here we generated 100 null cluster statistics, `num_permutations=100`, but we recommend running at least 200 permutations (500 to 1000 is best practice). 

In [ ]:
null_cluster_distribution = cluster_test.compute_null_cluster_stats(num_permutations=100)

We will compute the ***p*-value** associated with our true cluster statistics based on the null distributions we create using `cluster_significance_test()`.

In [ ]:
cluster_pvalue = cluster_test.cluster_significance_test(max_cluster_data,null_cluster_distribution) 
print(f'Positive cluster p-value: {cluster_pvalue[0]}')

Let's create a plot showing the **null distribution(s)** we generated, with our true cluster statistic overlaid on top.

In [ ]:
NeuroCluster.plot_null_distribution(null_cluster_distribution, max_cluster_data,cluster_pvalue,figsize=(6,5),dpi=125)

# Trying to replicate findings using MNE's built-in t-test functionality

When using a t-test to analyze a continuous variable, there is a risk of encountering false negatives (i.e., failing to detect a true effect), as demonstrated in this example. This issue arises because the t-test may not fully capture the complexity or structure of the data, leading to inaccurate conclusions. This underscores the advantages of NeuroCluster's approach, which provides a more robust method for capturing complex data structures and reducing the risk of false negatives.



In [ ]:
# Let's see if we get the same result as Neurocluster (a significant positive cluster meaning as error increases, encoding increases), using mne's built-in two sample t-test cluster function
import mne

# Let's take a median split for the error variable and assign it to high and low error variables (we need to discretize the error variable for the mne function)
error = sample_behav['error']
median_error = np.median(error)
low_error = tfr_data[error < median_error, :, :]    
high_error = tfr_data[error >= median_error, :, :]

# Let's run the mne two sample t-test cluster function
t_obs, clusters, cluster_pv, H0 = mne.stats.permutation_cluster_test([low_error, high_error], n_permutations=1000, tail=0)



In [ ]:
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_pv[i_c] <= 0.05:
        print(f"Cluster {i_c} p-value: {cluster_pv[i_c]}")
    else:
        print("No significant clusters found")

# Let's run the entire pipeline at once and save plots. 

In [ ]:
### NeuroCluster single electrode workflow: 

# Step 1: Create TFR_Cluster_Test Object
cluster_test  = NeuroCluster.TFR_Cluster_Test(tfr_data,predictor_data,target_var,demo_channel,alternative='two-sided')

# Step 2: Run TFR regression to extract beta coefficients for predictor of interest (permute_var) & tstats for each pixel in TFR. Determine which t-statistics are significant based on the critical t-value and save a thresholded t-statistic matrix.
betas, tstats = cluster_test.tfr_regression()
tstat_threshold = cluster_test.threshold_tfr_tstat(tstats)

# Step 3: Find largest cluster(s) and return the max cluster statistic(s) and cluster's  frequencies x times indices
max_cluster_data  = cluster_test.max_tfr_cluster(tstats,max_cluster_output='all')

# Step 4: Create null distribution of maximum cluster statistics from permuted data
null_cluster_distribution = cluster_test.compute_null_cluster_stats(num_permutations=100)

# Step 5: Use null cluster statistic distribution from permutations to compute non-parametric p value 
cluster_pvalue = cluster_test.cluster_significance_test(max_cluster_data,null_cluster_distribution) 

# Let's plot all the steps together and save the figures to a dictory that corresponds to the channel and predictor of interest. 
tcrit_plot,beta_plot,tstat_plot,cluster_plot,max_cluster_plot,null_distribution_plot = NeuroCluster.plot_neurocluster_results(betas,cluster_test, max_cluster_data, null_cluster_distribution, tstats, tstat_threshold,cluster_pvalue,freqs)

# Define the directory where you want to save the plots
output_directory = f'{results_dir}/{demo_channel}_{target_var}'

# Create the directory if it doesn't exist
NeuroCluster.create_directory(output_directory)

# Save plots to the output directory
NeuroCluster.save_plot_to_pdf(beta_plot, output_directory, 'beta_plot.pdf')
NeuroCluster.save_plot_to_pdf(tstat_plot, output_directory, 'tstat_plot.pdf')
NeuroCluster.save_plot_to_pdf(cluster_plot, output_directory, 'cluster_plot.pdf')
NeuroCluster.save_plot_to_pdf(max_cluster_plot, output_directory, 'max_cluster_plot.pdf')
NeuroCluster.save_plot_to_pdf(null_distribution_plot, output_directory, 'null_distribution_plot.pdf')